In [1]:
import os
#os.environ['IVOA_REGISTRY']="http://vao.stsci.edu/RegTAP/TapService.aspx"

import pyvo as vo
import warnings
# There are a number of relatively unimportant warnings that show up, so for now, suppress them:
warnings.filterwarnings("ignore", module="astropy.nddata.blocks.*")
warnings.filterwarnings("ignore", module="pyvo.utils.xml.*")
warnings.filterwarnings("ignore", module="urllib3.connectionpool.*")

#T Dower said:  " ForRegTAP, the preferred URL is now 
#  https://mast.stsci.edu/vo-tap/api/v0.1/registry. 
#  OAI-PMH is still on the old system."
navo_new_regtap = vo.dal.TAPService('https://mast.stsci.edu/vo-tap/api/v0.1/registry')
navo_old_regtap = vo.dal.TAPService('https://vao.stsci.edu/RegTAP/TapService.aspx')
gavo_regtap = vo.dal.TAPService('https://dc.zah.uni-heidelberg.de/__system__/tap/run')
euvo_regtap = vo.dal.TAPService('https://registry.euro-vo.org/regtap/tap')

from pyvo import registry
from astropy.coordinates import SkyCoord

# Registry Spring Cleaning notebook

Following up on Markus' [Confessions of a Registry Janitor](https://blog.g-vo.org/registry-a-janitor-speaks-out.html), I propose some regular checks of the metadata.  We already have checks of the validity of services, for instance, in the Operations group weather reports.  This would be compolementary.

## Check 1:  spot check numbers between different registries

What's the best way to get the current registries?  Testing one of them seems circular.  But the [RofR](https://rofr.ivoa.net) is still pointing to the old NAVO RegTAP.  OTOH, there's a bug in the new one.  

In [2]:
result = registry.search(datamodel="regtap").to_table()
print(result['ivoid','access_urls'])

            ivoid              ...
------------------------------ ...
        ivo://aip.gavo.org/tap ...
ivo://archive.stsci.edu/regtap ...
      ivo://esavo/registry/tap ...
         ivo://org.gavo.dc/tap ...
                ivo://purx/tap ...


In [3]:
def compare( query ):
    # Currently 
    #navo_regtap = vo.dal.TAPService('https://vao.stsci.edu/RegTAP/TapService.aspx')
    #navo_new_regtap = vo.dal.TAPService('https://mast.stsci.edu/vo-tap/api/v0.1/registry')
    navo_regtap = vo.dal.TAPService('https://mast.stsci.edu/vo-tap/api/v0.1/registry')
    gavo_regtap = vo.dal.TAPService('https://dc.zah.uni-heidelberg.de/__system__/tap/run')
    euvo_regtap = vo.dal.TAPService('https://registry.euro-vo.org/regtap/tap')

    for name,regtap in [('NAVO',navo_regtap),('GAVO',gavo_regtap),("EUVO",euvo_regtap)]:
        try:
            sias = regtap.search(query)
            print(f"{name} RegTAP finds {len(sias)}")
            strings=sias.to_table()['ivoid'] + ' "'+ sias.to_table()['cap_description']+'"'
        except Exception as e:
            print(f"{name} RegTAP gives error: {e}")       


So hardcode them for now

In [4]:
#  This syntax isn't processed correctly by NAVO's.  Use Markus' preferred.  
#compare("select count(*) as cnt from rr.capability where standard_id like '%sia%'")
compare("select * from rr.capability where standard_id like 'ivo://ivoa.net/std/sia%'")

NAVO RegTAP finds 477
GAVO RegTAP finds 477
EUVO RegTAP finds 474


In [5]:
compare("select * from rr.capability where standard_id like '%hips%'")

NAVO RegTAP finds 26
GAVO RegTAP finds 637
EUVO RegTAP finds 576


In [6]:
compare("select * from rr.capability where standard_id like '%cone%' and ivoid not like '%vizier%'")

NAVO RegTAP finds 1987
GAVO RegTAP finds 1956
EUVO RegTAP finds 1981


In [7]:
compare("select * from rr.capability where standard_id like '%cone%'  and ivoid like '%vizier%'")

NAVO RegTAP finds 24426


/Users/tjaffe/miniforge3/envs/navo-env/lib/python3.11/site-packages/pyvo/dal/query.py:341: DALOverflowWarning: Partial result set. Potential causes MAXREC, async storage space, etc.
  warn("Partial result set. Potential causes MAXREC, async storage space, etc.",


GAVO RegTAP finds 20000
EUVO RegTAP finds 29909


#### Check 1.1 look at the differences

In [8]:
query = "select * from rr.capability where standard_id like 'ivo://ivoa.net/std/sia%'"
def diffs(A, B, C, query):
    #  A is a tuple of ('name',service)
    resultsA = A[1].search(query).to_table()
    resultsB = B[1].search(query).to_table()
    resultsC = C[1].search(query).to_table()

    idsA=[r['ivoid']+'---'+r['cap_description'] for r in resultsA ]
    idsB=[r['ivoid']+'---'+r['cap_description'] for r in resultsB ]
    idsC=[r['ivoid']+'---'+r['cap_description'] for r in resultsC ]

    diffs1 = list( set(idsA) - set(idsB) )
    print(f"Found {len(diffs1)} in {A[0]} not in {B[0]}")
    diffs2 = list( set(idsA) - set(idsC) )
    print(f"Found {len(diffs2)} in {A[0]} not in {C[0]}")
    diffs3 = list( set(idsB) - set(idsC) )
    print(f"Found {len(diffs3)} in {B[0]} not in {C[0]}")

    diffs4 = list( set(idsB) - set(idsA) )
    print(f"Found {len(diffs4)} in {B[0]} not in {A[0]}")
    diffs5 = list( set(idsC) - set(idsA) )
    print(f"Found {len(diffs5)} in {C[0]} not in {A[0]}")
    diffs6 = list( set(idsC) - set(idsB) )
    print(f"Found {len(diffs6)} in {C[0]} not in {B[0]}")

    return [diffs1,diffs2,diffs3,diffs4,diffs5,diffs6]


In [9]:
difflists = diffs( ('NAVO',navo_old_regtap), ('GAVO',gavo_regtap), ('EUVO',euvo_regtap), 
      "select * from rr.capability where standard_id like 'ivo://ivoa.net/std/sia%'" )

Found 7 in NAVO not in GAVO
Found 9 in NAVO not in EUVO
Found 4 in GAVO not in EUVO
Found 7 in GAVO not in NAVO
Found 6 in EUVO not in NAVO
Found 1 in EUVO not in GAVO


In [10]:
print("Services in GAVO not in EUVO")
print("\n".join(difflists[2]))
print("Services in EUVO not in GAVO")
print("\n".join(difflists[5]))

Services in GAVO not in EUVO
ivo://vopdc.obspm/gepi/vopsat/srcj---
ivo://org.gavo.dc/lswscans/res/positions/siap---
ivo://vopdc.obspm/gepi/vopsat/esor---
ivo://vopdc.obspm/gepi/vopsat---
Services in EUVO not in GAVO
ivo://irsa.ipac/herschel/images/z0mgs_dust---


The hard part is then looking at those and understanding why.  What other information would we want to look at?

## Check 2:  UCDs 

#### Look at all UCDs in the Registry
- Check 2a:  are the UCDs valid according to astropy.io.votable.ucd.check_ucd

In [11]:
from astropy.io.votable.ucd import check_ucd
query="""
  select distinct ucd, count(*) as cnt
  from rr.table_column 
  group by ucd 
  order by cnt desc
  """
result = gavo_regtap.search(query)

all_ucds = result.to_table()
invalid_ucds = []
for i,u in enumerate(all_ucds['ucd'].data):
    if not check_ucd(u):
        invalid_ucds.append((u,all_ucds['cnt'][i]))
print(f"Found {len(invalid_ucds)} invalid UCDs")
print(f"  The top 10 bad UCD values by number of instances are")
x=[print(f"{c[0]:25}: {c[1]}") for c in invalid_ucds[0:10] ]

Found 79 invalid UCDs
  The top 10 bad UCD values by number of instances are
                         : 254736
??                       : 30342
vox:image_filesize       : 138
????                     : 70
vox:image_mjdateobs      : 59
image?                   : 49
vox:bandpass_id          : 47
vox:bandpass_hilimit     : 39
vox:bandpass_lolimit     : 39
vox:bandpass_refvalue    : 39


In [12]:
invalid_ucds_cv = []
for i,u in enumerate(all_ucds['ucd'].data):
    if not check_ucd(u,check_controlled_vocabulary=True):
        invalid_ucds_cv.append((u,all_ucds['cnt'][i]))
print(f"Found {len(invalid_ucds_cv)} that are not valid under UCD1+ controlled vocabulary")
print(f"  The top 10 bad UCD values by number of instances are")
[print(f"{c[0]:25}: {c[1]}") for c in invalid_ucds_cv[0:10] ]

Found 935 that are not valid under UCD1+ controlled vocabulary
  The top 10 bad UCD values by number of instances are
                         : 254736
??                       : 30342
error                    : 13825
code_misc                : 8538
phot_mag                 : 6291
fit_param                : 4505
obs.field                : 4334
number                   : 3090
id_number                : 2694
phot_intensity_adu       : 2512


[None, None, None, None, None, None, None, None, None, None]

#### UCDs at different publishers

Getting the publishers with the most resources in the Registry.  Let's check those.  

In [13]:
publishers = gavo_regtap.search("""
    select distinct role_ivoid, count(*) as cnt , role_name
    from rr.res_role 
    where base_role = 'publisher' and role_name != 'CDS'
    group by role_ivoid, role_name
    order by cnt desc
    """).to_table()[0:10]
publishers

role_ivoid,cnt,role_name
object,int32,object
ivo://nasa.heasarc/asd,1091,NASA/GSFC HEASARC
ivo://irsa.ipac/irsa,563,NASA/IPAC Infrared Science Archive
,244,The GAVO DC team
,217,Planetary Data System
ivo://wfau.roe.ac.uk,123,"WFAU, Institute for Astronomy, University of Edinburgh"
ivo://archive.stsci.edu/stsci-arc,96,Space Telescope Science Institute Archive
ivo://svo.cab,71,SVO CAB
ivo://noirlab.edu,65,NSF NOIRLab Astro Data Lab Team
,57,Paris Astronomical Data Centre


In [14]:
def validate_publishers(query, publist, badvallist, label):
    import pandas as pd #  Handy functions
    warnings.filterwarnings("ignore", message=".*This pattern is interpreted as a regular expression.*")
    for i,p in enumerate([pp.strip() for pp in publist['role_name'].data]):
        #  Look at all the metadata from this publisher
        print(f"\nlooking at publisher {p}")
        cnt_all = gavo_regtap.search(query.replace("xxxx",p))
        if len(cnt_all) != 0:  
            print(f"    publisher {p} publishes {len(cnt_all)} distinct values of {label}")
        else: 
            print(f"    publisher {p} publishes no column metadata (?)")
            continue #  ?
        df = pd.DataFrame(data={label:cnt_all.to_table()[label].data.data,"cnt":cnt_all.to_table()['cnt'].data.data})
      
        for c in badvallist:  #  invalid_ucds or invalid_ucds_cv (this is huge)
            #  c is a tuple of the string and the count
            if c[0]=='':  
                matches = df[label].astype(str).str.len() == 0
            elif '?' in c[0]:
                matches = df[label].str.contains("?",regex=False)
            else:
                matches = df[label] == c[0]
            cnt = df[matches]['cnt'].sum() # should only be one 
            if cnt == 0:  
                continue
            print(f"    value '{c[0]}' used {cnt} times")


query = f"""
        select ucd, count(*) as cnt from ( rr.res_role natural join rr.table_column )
        where role_name = 'xxxx'
        group by ucd 
        """
validate_publishers( query, publishers, invalid_ucds, "ucd")


looking at publisher NASA/GSFC HEASARC
    publisher NASA/GSFC HEASARC publishes 2451 distinct values of ucd
    value '' used 8596 times

looking at publisher NASA/IPAC Infrared Science Archive
    publisher NASA/IPAC Infrared Science Archive publishes no column metadata (?)

looking at publisher The GAVO DC team
    publisher The GAVO DC team publishes 830 distinct values of ucd
    value '' used 1373 times
    value 'vox:image_filesize' used 44 times
    value 'vox:image_mjdateobs' used 2 times

looking at publisher Planetary Data System
    publisher Planetary Data System publishes 41 distinct values of ucd
    value '' used 2604 times

looking at publisher WFAU, Institute for Astronomy, University of Edinburgh
    publisher WFAU, Institute for Astronomy, University of Edinburgh publishes 855 distinct values of ucd
    value '' used 301789 times
    value '??' used 60549 times
    value '????' used 60549 times
    value 'image?' used 60549 times
    value '???' used 60549 times
  

In [15]:
culprits = []
for i,u in enumerate(all_ucds['ucd'].data):
    if not check_ucd(u,check_controlled_vocabulary=True):
        culprits.append((u,all_ucds['cnt'][i]))
print(f"Found {len(culprits)} that are not valid under UCD1+ controlled vocabulary")
print(f"  The top 10 bad UCD values by number of instances are")
x=[print(f"{c[0]:25}: {c[1]}") for c in culprits[0:10] ]

Found 935 that are not valid under UCD1+ controlled vocabulary
  The top 10 bad UCD values by number of instances are
                         : 254736
??                       : 30342
error                    : 13825
code_misc                : 8538
phot_mag                 : 6291
fit_param                : 4505
obs.field                : 4334
number                   : 3090
id_number                : 2694
phot_intensity_adu       : 2512


### Check 3:  authors

Have
* Last F.
* Last F., Last2 F.
* Last, F.
* F. Last, Last2. F.

At least where there are commas they are used to separate two authors, rather than "Last, F" or something.

In [16]:
names = gavo_regtap.search("select distinct role_name, count(*) as cnt from rr.res_role where base_role = 'creator' group by role_name").to_table()
names

/Users/tjaffe/miniforge3/envs/navo-env/lib/python3.11/site-packages/pyvo/dal/query.py:341: DALOverflowWarning: Partial result set. Potential causes MAXREC, async storage space, etc.
  warn("Partial result set. Potential causes MAXREC, async storage space, etc.",


role_name,cnt
object,int32
"Lagrange A.-M.,Langlois M.",1
"Burstein D.,Bohlin R.C.",1
DAVILA H.,1
"Dumusque X.,Fulton B.J.",1
"Queloz D.,Rauer H.",1
Hillwig T.C.,7
Ranadive P.,1
Lalitha S.,9
Moriarty-Schieven G.,6


### Check 4:  subjects and the UAT

In [17]:
subjects = gavo_regtap.search("select res_subject, count(*) as cnt from rr.res_subject group by res_subject order by cnt desc").to_table()
subjects

res_subject,cnt
object,int32
visible-astronomy,7270
galaxies,4355
infrared-photometry,4333
spectroscopy,4294
photometry,3837
radial-velocity,2856
surveys,2761
redshifted,2669
variable-stars,2033


In [18]:
import urllib.request, json 
with urllib.request.urlopen("https://raw.githubusercontent.com/astrothesaurus/UAT/master/UAT.json") as url:
    uat = json.load(url)

In [19]:
#  Generator that goes through the nested JSON and looks for a key anywhere down in it
def item_generator(json_input, lookup_key):
    if isinstance(json_input, dict):
        for k, v in json_input.items():
            if k == lookup_key:
                yield v
            else:
                yield from item_generator(v, lookup_key)
    elif isinstance(json_input, list):
        for item in json_input:
            yield from item_generator(item, lookup_key)

In [20]:
uat_name_list = [x.lower() for x in item_generator(uat,'name')]
print(f"Found {len(uat_name_list)} names in the UAT")
print(uat_name_list[0:10])

Found 4335 names in the UAT
['astrophysical processes', 'astrophysical magnetism', 'cosmic magnetic fields theory', 'emerging flux tubes', 'magnetic fields', 'geomagnetic fields', 'magnetic anomalies', 'primordial magnetic fields', 'gravitation', 'relativity']


In [21]:
invalid_subjects = []
correct_subjects = []
for i,s in enumerate(subjects['res_subject'].data):
    if s.lower() in uat_name_list:
        correct_subjects.append((s,subjects['cnt'][i]))
    else:
        invalid_subjects.append((s,subjects['cnt'][i]))
print(f"Found {len(invalid_subjects)} Registry res_subject entries \
that are not in the UAT and {len(correct_subjects)} that are.")
print(f"  The top 10 bad subject values by number of instances are")
x=[print(f"{c[0]}: {c[1]}") for c in invalid_subjects[0:10] ]

Found 1013 Registry res_subject entries that are not in the UAT and 240 that are.
  The top 10 bad subject values by number of instances are
visible-astronomy: 7270
infrared-photometry: 4333
radial-velocity: 2856
variable-stars: 2033
Wide-band photometry: 1937
multiple-stars: 1869
x-ray-sources: 1723
open-star-clusters: 1677
chemical-abundances: 1611
infrared-sources: 1433


In [22]:
import re
result = [u for u in uat_name_list if re.search("^star.*",u)]
print(f"Found {len(result)} matches to 'star' such as")
print(result[0:10])

Found 17 matches to 'star' such as
['star-planet interactions', 'starburst galaxies', 'starburst galaxies', 'starburst galaxies', 'star atlases', 'star counts', 'star counts', 'star lore', 'starspots', 'starspots']


In [23]:
query = f"""
        select top 10 res_subject, count(*) as cnt from ( rr.res_role natural join rr.res_subject )
        where role_name = 'xxxx'
        group by res_subject order by cnt desc
        """
validate_publishers( query, publishers, invalid_subjects, "res_subject")


looking at publisher NASA/GSFC HEASARC
    publisher NASA/GSFC HEASARC publishes 10 distinct values of res_subject
    value 'Survey Source' used 654 times
    value 'Observation' used 84 times
    value 'Star' used 70 times
    value 'Galaxy' used 25 times
    value 'GRB' used 31 times
    value 'AGN' used 22 times
    value 'Cluster of Galaxies' used 15 times
    value 'Optical Counterpart' used 11 times
    value 'XRB' used 11 times

looking at publisher NASA/IPAC Infrared Science Archive
    publisher NASA/IPAC Infrared Science Archive publishes 10 distinct values of res_subject
    value '' used 137 times
    value 'extragalactic survey' used 101 times
    value 'all sky survey' used 58 times
    value 'high redshift galaxies' used 17 times

looking at publisher The GAVO DC team
    publisher The GAVO DC team publishes 10 distinct values of res_subject
    value 'proper-motions' used 31 times
    value 'milky-way-galaxy' used 13 times
    value 'virtual-observatories' used 75 tim

### Check 5:  concepts

In [24]:
reg_uat_concept_list = gavo_regtap.search("select distinct uat_concept from rr.subject_uat").to_table()["uat_concept"].data
print(f"There are {len(reg_uat_concept_list)} distinct uat_concept values in the registry's subject_uat table")

There are 471 distinct uat_concept values in the registry's subject_uat table


In [25]:
bad=[]
for c in reg_uat_concept_list:
    # lower case and replace - with space
    if c.lower().replace("-"," ") not in uat_name_list:
        bad.append(c)
print(f"There are {len(bad)} concepts not found in the UAT such as:")
print(bad[0:10])

There are 47 concepts not found in the UAT such as:
['active-galactic-nuclei ', 'astrl', 'astronomical-simulations ', 'early-type-galaxies', 'early-type-stars', 'earth-planet', 'earth-planet-', 'exoplanet-atmospheric-composition', 'gamma-ray-astronomy', 'gamma-ray-bursts']


In [26]:
query = f"""
        select top 10 uat_concept, count(*) as cnt from ( rr.res_role natural join rr.subject_uat )
        where role_name = 'xxxx'
        group by uat_concept order by cnt desc
        """
validate_publishers( query, publishers, bad, "uat_concept")


looking at publisher NASA/GSFC HEASARC
    publisher NASA/GSFC HEASARC publishes 10 distinct values of uat_concept

looking at publisher NASA/IPAC Infrared Science Archive
    publisher NASA/IPAC Infrared Science Archive publishes 10 distinct values of uat_concept

looking at publisher The GAVO DC team
    publisher The GAVO DC team publishes 10 distinct values of uat_concept

looking at publisher Planetary Data System
    publisher Planetary Data System publishes 6 distinct values of uat_concept

looking at publisher WFAU, Institute for Astronomy, University of Edinburgh
    publisher WFAU, Institute for Astronomy, University of Edinburgh publishes 7 distinct values of uat_concept

looking at publisher Space Telescope Science Institute Archive
    publisher Space Telescope Science Institute Archive publishes 10 distinct values of uat_concept

looking at publisher SVO CAB
    publisher SVO CAB publishes 8 distinct values of uat_concept

looking at publisher NSF NOIRLab Astro Data Lab 

## To be expanded.  Now what to do with this?  

* Report cross-checks between registries to their admins.  
* Compile a report of issues as above and advertise at IVOA Interop's Registry (or Ops?) session.  
* Compile a report of issues found for each publisher and email them yearly to request updates.  


## Scratch 